In [71]:
import pandas as pd
import numpy as np

In [72]:
def GetPosition (array):
    if isinstance(array,np.ndarray):
        if array.__contains__('C'):
            return 'C'
        if array.__contains__('2B'):
            return '2B'
        if array.__contains__('SS'):
            return 'SS'
        if array.__contains__('3B'):
            return '3B'
        if array.__contains__('1B'):
            return '1B'
        if array.__contains__('OF'):
            return 'OF'
    else: 
        return '1B'

In [73]:
atcbat = pd.read_csv("batters.csv")

In [74]:
atcpit = pd.read_csv("pitchers.csv")

In [75]:
hittersmean = atcbat
pitchersmean = atcpit

In [76]:
hittersmean

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,OBP,SLG,OPS,wOBA,ADP,-1,InterSD,InterSK,IntraSD,playerid
0,Juan Soto,WSN,153,657,539,164,34,3,38,110,...,0.424,0.591,1.015,0.426,3.5,NaN,3.79,0.29,0.68,20123
1,Mike Trout,LAA,149,652,524,149,27,4,44,112,...,0.416,0.602,1.018,0.421,6.0,NaN,2.26,-0.56,1.11,10155
2,Freddie Freeman,ATL,154,674,575,172,38,2,32,106,...,0.398,0.543,0.941,0.396,13.1,NaN,1.90,-0.11,1.20,5361
3,Ronald Acuna Jr.,ATL,149,657,562,156,29,3,39,118,...,0.378,0.546,0.923,0.390,1.6,NaN,4.04,0.10,0.94,18401
4,Christian Yelich,MIL,148,648,546,150,29,3,35,103,...,0.386,0.530,0.916,0.387,10.7,NaN,3.47,-1.43,0.41,11477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,Rodolfo Castro,NaN,9,34,32,6,1,0,1,3,...,0.240,0.315,0.555,0.242,999.0,NaN,2.99,2.50,1.85,sa917362
639,Joe Hudson,PIT,10,28,25,5,1,0,0,2,...,0.256,0.282,0.538,0.242,999.0,NaN,1.68,0.18,1.89,14412
640,Billy Hamilton,CHW,14,45,41,9,1,0,0,6,...,0.274,0.255,0.528,0.240,599.7,NaN,3.86,3.41,1.96,10199
641,Ryan Goins,ATL,26,34,30,5,1,0,1,4,...,0.250,0.270,0.520,0.237,999.0,NaN,0.00,0.00,1.78,9807


In [77]:
hittersmean['OBPVal'] = ((hittersmean['PA']*hittersmean['OBP']++2037.4)/(hittersmean['PA']+6100)-.334)/.0016
hittersmean['AVGVal'] = ((hittersmean['H']+1768)/(hittersmean['AB']+6100)-.267)/.0024
hittersmean['value'] = hittersmean['OBPVal']+(hittersmean['R']/8.92+hittersmean['HR']/7.5+hittersmean['RBI']/8.92+hittersmean['SB']/6.0)

In [78]:
#https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=1&season=2020&month=0&season1=2020&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=
positions = pd.read_csv("positions.csv")

In [79]:
positions['Pos']=positions['Pos'].replace(['LF','CF','RF'],'OF')


In [80]:
positions_unique=positions.loc[(positions['G']>=10) | (positions['GS']>=5)].groupby('playerid')['Pos'].unique()


In [81]:
positions_unique.index=positions_unique.index.map(str)

In [82]:
hitterspos=pd.merge(hittersmean,positions_unique,how='left',left_on='playerid',right_index=True)


In [83]:
hitterspos

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,ADP,-1,InterSD,InterSK,IntraSD,playerid,OBPVal,AVGVal,value,Pos
0,Juan Soto,WSN,153,657,539,164,34,3,38,110,...,3.5,NaN,3.79,0.29,0.68,20123,5.469328,10.003201,37.757220,[OF]
1,Mike Trout,LAA,149,652,524,149,27,4,44,112,...,6.0,NaN,2.26,-0.56,1.11,10155,4.948904,9.334239,37.036796,[OF]
2,Freddie Freeman,ATL,154,674,575,172,38,2,32,106,...,13.1,NaN,1.90,-0.11,1.20,5361,3.979923,9.848627,33.070955,[1B]
3,Ronald Acuna Jr.,ATL,149,657,562,156,29,3,39,118,...,1.6,NaN,4.04,0.10,0.94,18401,2.673894,9.084234,36.361936,[OF]
4,Christian Yelich,MIL,148,648,546,150,29,3,35,103,...,10.7,NaN,3.47,-1.43,0.41,11477,3.120925,8.997768,32.203137,[OF]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,Rodolfo Castro,NaN,9,34,32,6,1,0,1,3,...,999.0,NaN,2.99,2.50,1.85,sa917362,-0.325644,9.292509,0.480335,NaN
639,Joe Hudson,PIT,10,28,25,5,1,0,0,2,...,999.0,NaN,1.68,0.18,1.89,14412,-0.222748,9.362245,0.337790,[C]
640,Billy Hamilton,CHW,14,45,41,9,1,0,0,6,...,599.7,NaN,3.86,3.41,1.96,10199,-0.274614,9.319397,1.401022,[OF]
641,Ryan Goins,ATL,26,34,30,5,1,0,1,4,...,999.0,NaN,0.00,0.00,1.78,9807,-0.291001,9.263866,0.739193,NaN


In [84]:
hitterspos['BestPos']=hitterspos['Pos'].apply(GetPosition)

hitterspos['rank']=hitterspos.groupby('BestPos')['value'].rank(ascending=False)
hitterspos['utilrank']=hitterspos['value'].rank(ascending=False)


In [85]:
###customized league replacement value
replacementvalue=hitterspos.groupby('BestPos').value.nlargest(10).groupby('BestPos').min().reset_index(name='PosValue')
replacementvalue['UtilValue']=float(hitterspos.loc[hitterspos['utilrank']==80]['value'])

replacementvalue['ReplacementValue']=pd.DataFrame([replacementvalue['UtilValue'], replacementvalue['PosValue']]).min()


In [86]:
replacementvalue

,BestPos,PosValue,UtilValue,ReplacementValue
0,1B,24.925212,21.81752,21.817520
1,2B,22.151789,21.81752,21.817520
2,3B,22.116784,21.81752,21.817520
3,C,13.373929,21.81752,13.373929
4,OF,27.215006,21.81752,21.817520
5,SS,23.662320,21.81752,21.817520


In [87]:
hittersfinal=hitterspos.reset_index().merge(replacementvalue,on='BestPos')

hittersfinal['VORP']=hittersfinal['value']-hittersfinal['ReplacementValue']


In [88]:
hittersfinal

,index,Name,Team,G,PA,AB,H,2B,3B,HR,...,AVGVal,value,Pos,BestPos,rank,utilrank,PosValue,UtilValue,ReplacementValue,VORP
0,0,Juan Soto,WSN,153,657,539,164,34,3,38,...,10.003201,37.757220,[OF],OF,1.0,1.0,27.215006,21.81752,21.817520,15.939701
1,1,Mike Trout,LAA,149,652,524,149,27,4,44,...,9.334239,37.036796,[OF],OF,2.0,2.0,27.215006,21.81752,21.817520,15.219276
2,3,Ronald Acuna Jr.,ATL,149,657,562,156,29,3,39,...,9.084234,36.361936,[OF],OF,3.0,3.0,27.215006,21.81752,21.817520,14.544416
3,4,Christian Yelich,MIL,148,648,546,150,29,3,35,...,8.997768,32.203137,[OF],OF,6.0,9.0,27.215006,21.81752,21.817520,10.385617
4,5,Bryce Harper,PHI,151,654,533,140,29,2,36,...,8.605269,33.389402,[OF],OF,5.0,6.0,27.215006,21.81752,21.817520,11.571882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,619,Drew Butera,NaN,21,63,56,12,2,0,1,...,9.228666,0.944848,[C],C,67.0,566.0,13.373929,21.81752,13.373929,-12.429082
639,624,Sandy Leon,MIA,18,63,56,11,2,0,1,...,9.160981,0.864685,[C],C,69.0,580.0,13.373929,21.81752,13.373929,-12.509245
640,636,Jeff Mathis,NaN,16,55,50,9,2,0,2,...,9.142954,0.896273,[C],C,68.0,572.0,13.373929,21.81752,13.373929,-12.477657
641,639,Joe Hudson,PIT,10,28,25,5,1,0,0,...,9.362245,0.337790,[C],C,76.0,641.0,13.373929,21.81752,13.373929,-13.036139


In [89]:

########################
#CHANGE THE COLUMNS BELOW BASED ON SGP VALUES FOR YOUR LEAGUE
pitchersmean['ERA']=((525.55+pitchersmean['ER'])*9/(1350.0+pitchersmean['IP'])-3.5)/-0.1062
pitchersmean['WHIP'] =((1620.0+pitchersmean['H']+pitchersmean['BB'])/(1350.0+pitchersmean['IP'])-1.2)/-0.0201
pitchersmean['value'] = pitchersmean['ERA']+pitchersmean['WHIP']+(pitchersmean['W']/4.05+pitchersmean['K/9']*pitchersmean['IP']/(55.0*9.0)+(pitchersmean['SV']+pitchersmean['HLD'])/10.0)
########################

pitchersmean['utilrank']=pitchersmean['value'].rank(ascending=False)

###customized league replacement value
pitchersmean['ReplacementValue']=float(pitchersmean.loc[pitchersmean['utilrank']==120]['value'])

pitchersmean['VORP']=pitchersmean['value']-pitchersmean['ReplacementValue']


In [90]:
pitchersmean

,Name,Team,W,L,SV,HLD,ERA,GS,G,IP,...,ADP,-1,InterSD,InterSK,IntraSD,playerid,value,utilrank,ReplacementValue,VORP
0,Liam Hendriks,CHW,4,3,33,3,0.306226,0,64,66.0,...,55.2,NaN,2.13,0.18,1.79,3548,6.994528,16.0,3.500496,3.494032
1,Edwin Diaz,NYM,3,3,30,4,0.200111,0,65,64.0,...,73.4,NaN,4.98,-0.27,1.63,14710,6.446546,19.0,3.500496,2.946050
2,Jacob deGrom,NYM,14,7,0,0,0.885745,30,31,192.0,...,4.8,NaN,6.04,-1.09,1.97,10954,10.287680,1.0,3.500496,6.787184
3,James Karinchak,CLE,3,2,24,5,0.080244,0,65,64.0,...,95.1,NaN,3.16,0.02,1.38,20151,5.736641,29.0,3.500496,2.236145
4,Devin Williams,MIL,4,3,6,20,0.292513,0,63,68.0,...,181.1,NaN,4.57,0.77,0.87,15816,5.783349,27.0,3.500496,2.282853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,Nestor Cortes,NYY,0,0,0,0,-0.114146,0,3,7.0,...,999.0,NaN,3.31,-1.97,1.06,17874,-0.052746,735.0,3.500496,-3.553242
762,Dillon Peters,LAA,1,2,0,0,-0.209134,2,18,21.0,...,999.0,NaN,4.35,-0.46,0.83,18790,0.264505,601.0,3.500496,-3.235991
763,Albert Abreu,NYY,0,0,0,1,-0.114146,0,6,7.0,...,999.0,NaN,5.48,-2.88,1.03,17485,0.022470,711.0,3.500496,-3.478026
764,Justin Grimm,NaN,2,1,0,0,-0.618515,0,26,27.0,...,999.0,NaN,2.56,0.78,0.49,11720,-0.139461,752.0,3.500496,-3.639957


In [91]:
final=pd.concat([hittersfinal,pitchersmean])


In [92]:
final.sort_values('VORP', ascending=False)

,index,Name,Team,G,PA,AB,H,2B,3B,HR,...,SV,HLD,ERA,GS,IP,ER,WHIP,K/9,BB/9,FIP
0,0.0,Juan Soto,WSN,153,657.0,539.0,164,34.0,3.0,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Mike Trout,LAA,149,652.0,524.0,149,27.0,4.0,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Ronald Acuna Jr.,ATL,149,657.0,562.0,156,29.0,3.0,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7.0,Mookie Betts,LAD,148,666.0,577.0,167,35.0,4.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,8.0,Fernando Tatis Jr.,SDP,147,648.0,574.0,162,29.0,6.0,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,597.0,Jose Briceno,COL,6,22.0,20.0,4,1.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,592.0,Cheslor Cuthbert,CIN,13,27.0,25.0,5,1.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324,507.0,Omar Estevez,LAD,5,22.0,20.0,5,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,608.0,Bubba Starling,KCR,7,23.0,22.0,5,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
final.sort_values('VORP', ascending=False).nlargest(200,['VORP']).to_excel("Atc.xlsx")